## Импорт библиотек

In [2]:
# pip install fitz

In [3]:
# pip install PyMuPDF

In [4]:
# pip install pymupdf pandas tqdm

In [5]:
# pip install PyPDF2

In [6]:
import os
import pandas as pd
import re
import fitz  # PyMuPDF
from tqdm import tqdm
from PyPDF2 import PdfReader

In [7]:
def get_metadata(text):
    """Извлекает метаданные из текста PDF"""
    metadata = {
        "title": "",
        "author": "",
        "tags": [],
        "hubs": []
    }
    
    # Извлечение заголовка (первая строка после некоторых пустых строк)
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    if lines:
        metadata["title"] = lines[0]
    
    # Поиск автора (формат "Автор · дата")
    # author_pattern = r"([A-Za-zА-Яа-яЁё0-9_]+)\s+[\d]+\s+[а-я]+\s+в\s+[\d:]+"
    # author_pattern = r"\d+ мин\s+([A-Za-zА-Яа-яЁё0-9_]+)\s*\n.*\n"
    author_pattern = ""



    
    author_after_time = re.search(r"\d+\s*мин\s*\n\s*([A-Za-zА-Яа-яЁё0-9_]+)\s*\n", text)
    if author_after_time:
        author_pattern = r"\d+\s*мин\s*\n\s*([A-Za-zА-Яа-яЁё0-9_]+)\s*\n"
    
    # Паттерн 2: Упоминание в формате "@Имя Пользователь" (пример: "@Kual Пользователь")
    author_mention = re.search(r"@([A-Za-zА-Яа-яЁё0-9_]+)\s+Пользователь", text)
    if author_mention:
        author_pattern = r"@([A-Za-zА-Яа-яЁё0-9_]+)\s+Пользователь"
    
    # Паттерн 3: Название компании/бота в отдельной строке (пример: "getmatch")
    company_name = re.search(r"^\s*([A-Za-zА-Яа-яЁё0-9_]{4,})\s*$", text, re.MULTILINE)
    if company_name:
        author_pattern = r"^\s*([A-Za-zА-Яа-яЁё0-9_]{4,})\s*$"
    
    # Паттерн 4: В строке с рейтингом/кармой (пример: "14 Карма\n32.6 Рейтинг\nКурогло Александр @Kual")
    author_line = re.search(r"Рейтинг\s*\n[^\n]*@([A-Za-zА-Яа-яЁё0-9_]+)", text)
    if author_line:
        author_pattern = r"Рейтинг\s*\n[^\n]*@([A-Za-zА-Яа-яЁё0-9_]+)"




    
    match = re.search(author_pattern, text)
    if match:
        metadata["author"] = match.group(1)
    
    # Извлечение тегов (строка, начинающееся с "Теги:" или "Тети:")
    tags_pattern = r"(?:Теги|Тети):\s*([^\n]+)"
    tags_match = re.search(tags_pattern, text)
    if tags_match:
        metadata["tags"] = [tag.strip() for tag in tags_match.group(1).split(',')]
    
    # Извлечение хабов (ищем строку, начинающуюся с "Хабы:")
    hubs_pattern = r"Хабы:\s*([^\n]+)"
    hubs_match = re.search(hubs_pattern, text)
    if hubs_match:
        metadata["hubs"] = [hub.strip() for hub in hubs_match.group(1).split(',')]
    
    return metadata

In [8]:
def parse_pdf_content(text, filename):
    """Анализирует содержимое PDF и извлекает нужные данные"""
    # Извлекаем метаданные
    metadata = get_metadata(text)
    
    # Основной контент (исключаем первые строки, которые могут быть метаданными)
    content_lines = text.split('\n')[10:]  # Пропускаем первые 10 строк
    content = "\n".join([line for line in content_lines if line.strip()])
    
    return {
        "file_name": filename[:len(filename)-4],
        "title": metadata["title"],
        "author": metadata["author"],
        "tags": ", ".join(metadata["tags"]),
        "hubs": ", ".join(metadata["hubs"]),
        "content": content,
        "char_count": len(text),
        "word_count": len(text.split())
    }

In [9]:
# получаем текст из пдф
def extract_text_from_pdf(file_path):    
    text = ""
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

In [10]:
def process_pdf_files(folder_path):
    """Обрабатывает все PDF файлы в указанной папке"""
    # Находим все PDF файлы
    pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        print("PDF файлы не найдены в указанной папке")
        return None
    
    print(f"Найдено PDF файлов: {len(pdf_files)}")
    print("Список PDF файлов:")
    for i, pdf in enumerate(pdf_files, 1):
        print(f"{i}. {pdf}")
    
    # Парсим каждый PDF файл
    data = []
    for pdf_file in tqdm(pdf_files, desc="Обработка PDF файлов"):
        file_path = os.path.join(folder_path, pdf_file)
        text = extract_text_from_pdf(file_path)
        parsed_data = parse_pdf_content(text, pdf_file)
        data.append(parsed_data)
    
    return data

In [11]:
def save_to_csv(data, output_file):
    """Сохраняет данные в CSV файл"""
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Данные сохранены в {output_file}")

In [27]:
pdf_folder = "PDF2"  # путь

# Обрабатываем PDF файлы
parsed_data = process_pdf_files(pdf_folder)

if parsed_data:
    # Создаем DataFrame
    df = pd.DataFrame(parsed_data)
    print("\nВывод:")
    print(df.head())
    
    
    # Сохраняем в CSV
    output_csv = "parsing_pdf_csv.csv"
    # save_to_csv(parsed_data, output_csv)


Вывод:
                                           file_name  \
0  AI-агенты в реальном мире_ почему они не работ...   
1  Cocoapods, Carthage, SPM как выбрать менеджер ...   
2  Deutsche Telekom и Perplexity объявили о новом...   
3              imgonline-com-ua-site2pdfB28eKGLy6EjA   
4  OSINT & Hacking — как работает фишинг для нель...   

                                               title         author  \
0  Меня з овут Алек сандр, я COO в SaaS-пла тформ...           Kual   
1                                             117.94    redmadrobot   
2  Еще до на чала MWC в Барсе лоне был о очевидно...  technokratiya   
3                                              71.07                  
4                                              71.07                  

                                                tags  \
0  ai-агент, llm-мо дели, llm, инженерия наде жно...   
1                                      red_mad_robot   
2  смар тфоны, ии, искусственный инт еллек т, моб...   
3   

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   file_name   25 non-null     object
 1   title       25 non-null     object
 2   author      25 non-null     object
 3   tags        25 non-null     object
 4   hubs        25 non-null     object
 5   content     25 non-null     object
 6   char_count  25 non-null     int64 
 7   word_count  25 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 1.7+ KB


In [33]:
display(df.head(len(df)))

,file_name,title,author,tags,hubs,content,char_count,word_count
0,AI-агенты в реальном мире_ почему они не работ...,"Меня з овут Алек сандр, я COO в SaaS-пла тформ...",Kual,"ai-агент, llm-мо дели, llm, инженерия наде жно...","Искусственный инт еллек т, Управ ление ра зраб...",Интерес к аг ентам прояв ляется со всех сторон...,29018,5384
1,"Cocoapods, Carthage, SPM как выбрать менеджер ...",117.94,redmadrobot,red_mad_robot,Блог компании red_mad_robot,менеджер зависимостей в iOS\n179\nБлог компани...,27968,3696
2,Deutsche Telekom и Perplexity объявили о новом...,Еще до на чала MWC в Барсе лоне был о очевидно...,technokratiya,"смар тфоны, ии, искусственный инт еллек т, моб...","Искусственный инт еллек т, Смар тфоны",стоимостью менее 1 000 долларов\n4 мин 419\...,6675,1278
3,imgonline-com-ua-site2pdfB28eKGLy6EjA,71.07,,взлом,Блог компании Cloud4Y,Cloud4Y\n5 часов назад\nOSINT & Hacking — как ...,5034,710
4,OSINT & Hacking — как работает фишинг для нель...,71.07,,взлом,Блог компании Cloud4Y,5 часов назад\nПростой\n3 мин\nCloud4Y\nOSINT ...,4997,704
5,Yandex N.V. запретили в течение пяти лет созда...,Нидер ландск ая компания Y andex N.V . по ит о...,,"яндек с, yandex n.v ., разделение бизнеса","Управ ление проек тами, Бизнес-мо дели, Финанс...",сделку по про даже «Яндек са» за 475 млр д руб...,2234,416
6,Быстрое начало работы с Gitlab CICD,4.29,,ci/cd,Блог компании Southbridge,"CI/CD, чтобы быстро вникнуть в основы технолог...",9879,1336
7,Генеральный директор Mozilla покинула свой пос...,"Митче лл Бейк ер, гендирек тор Mozilla с 2020 ...",AnnieBronson,"mozilla, mozilla foundation","IT-компании, Управ ление перс онал ом, Карьера...",учре дила лиц ензию Mozilla Public License.\nA...,6204,1052
8,За что безопасники будут гореть в аду_ _ Хабр,Для прив лечения внимания расск ажу ист орию. ...,,"свобода, разрабо тка, разрабо тка прил ожений,","Разрабо тка мобильных прил ожений, Разрабо тка...","навсегда. Как ое-то знак омое чув ство, не т а...",7260,1431
9,Заезжаем в Kotlin Multiplatform. Но какой цено...,"Прив ет! Меня з овут Денис, я занимаюсь к омме...",,,,Заезжаем в Kotlin Multiplatform. Но какой ...,24724,4999
